In [ ]:
import torch
# import os
# import argparse
import torch.nn.functional as F
# import ipdb
# import numpy as np
import logging

from torch import nn
from data.preprocessing import AudioProcessor
from data.audio_transforms import MelScale
from data.loaders import get_data_loader
# from tqdm import tqdm, trange
from utils.utils import mkdir_in_path, GPU_is_available
from torch.utils.data import DataLoader

from os.path import dirname, realpath, join
from datetime import datetime
# from sklearn.metrics import confusion_matrix, classification_report
# from utils.utils import read_json
from datetime import datetime
from evaluation.inception_network import SpectrogramInception3
# from pprint import pprint

In [ ]:
name = 'name'
path = 'evaluation/inception_models'
config = {
    "name": "inception train on footsteps",
    "comments": "inception train on footsteps",
    "output_path": "../output_folder",
    "loader_config": {
        "dbname": "footsteps",
        "data_path": "../../_footsteps_data/zapsplat_misc_shoes_misc_surfaces_inception_network/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 0.9
    },
    "transform_config": {
        "transform": "stft",
        "fade_out": True,
        "fft_size": 1024,
        "win_size": 1024,
        "n_frames": 64,
        "hop_size": 256,
        "log": False,
        "ifreq": False,
        "sample_rate": 16000,
        "audio_length": 8192
    }
}

batch_size = 1
n_epoch = 100

In [ ]:
output_path = mkdir_in_path(path, 'inception_models')
output_file = join(output_path, f"{name}_{datetime.now().strftime('%Y-%m-%d')}.pt")
output_log = join(output_path, f"{name}_{datetime.now().strftime('%Y-%m-%d')}.log")
logging.basicConfig(filename=output_log, level=logging.INFO)

loader_config = config['loader_config']
print("-- TRAIN INCEPTION MODEL: loader_config --")
print(loader_config)
print()

transform_config = config['transform_config']
print("-- TRAIN INCEPTION MODEL: transform_config --")
print(transform_config)
print()

transform = transform_config['transform']
print("-- TRAIN INCEPTION MODEL: transform --")
print(transform)
print()

dbname = loader_config['dbname']
print("-- TRAIN INCEPTION MODEL: dbname --")
print(dbname)
print()

loader_module = get_data_loader(dbname)
print("-- TRAIN INCEPTION MODEL: loader_module --")
print(loader_module)
print()

In [ ]:
processor = AudioProcessor(**transform_config)

In [ ]:
loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **loader_config)

In [ ]:
mel = MelScale(sample_rate=transform_config['sample_rate'],
                fft_size=transform_config['fft_size'],
                n_mel=transform_config.get('n_mel', 256),
                rm_dc=True)

In [ ]:
val_data, val_labels = loader.get_validation_set()
print(val_data.shape)
val_data = val_data[:, 0:1]
print(val_data.shape)
print(val_labels[:5])

In [ ]:
data_loader = DataLoader(loader,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=2)

In [ ]:
data_loader.batch_size

In [ ]:
print("-- TR DATA --")
print(len(loader.data))
print()
print("-- VAL DATA --")
print(len(loader.val_data))
print()
print("-- NUM CLASSES --")
num_classes = len(set(loader.metadata))
print(num_classes)
print()

In [ ]:
device = "cuda" if GPU_is_available() else "cpu"

inception_model = nn.DataParallel(
        SpectrogramInception3(num_classes, aux_logits=False))
inception_model.to(device)

optim = torch.optim.Adam(filter(lambda p: p.requires_grad, inception_model.parameters()),
                    betas=[0, 0.99], lr=0.001)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
data_iter = iter(data_loader)
input, labels = data_iter.next()

In [ ]:
input.shape

In [ ]:
labels